In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import csv

In [8]:
main_url = 'https://www.iranketab.ir/book'

In [9]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}


In [10]:
response = requests.get(url=main_url , headers=headers)
print(response.status_code)

200


In [11]:
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
page_number = 1  # Start with the first page

request_count = 0

session = requests.Session()
visited_urls = set()

with open('BooksUrls.csv', 'w', newline='') as csvfile:
    url_writer = csv.writer(csvfile)

    while True:
        current_page_url = f"{main_url}?pagenumber={page_number}&pagesize=20"
        response = session.get(current_page_url)

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        allBooks = soup.find_all("a", class_="product-item-link")
        
        for book_url in allBooks:
            href = book_url.get("href")
            if href:
                full_url = f'{main_url}{href}'
                if full_url not in visited_urls:
                    visited_urls.add(full_url)
                    url_writer.writerow([full_url])
                    request_count += 1

                    if request_count % 500 == 0:
                        time.sleep(1)

        next_page_link = soup.find("a", {"data-page-no": str(page_number + 1)})

        if next_page_link:
            page_number += 1
        else:
            break

In [19]:
#csv_file = 'final_links.csv'

In [21]:
""" BookURLs = pd.read_csv('BooksUrls.csv', header=None)
urls = BookURLs[0].tolist()
urls = [url.replace("/book", "", 1) for url in urls]
link = pd.DataFrame({'final_books_Url': urls}) """

In [22]:
#link.to_csv('final_books_Url.csv', index=False)

In [25]:
BookURLs = pd.read_csv('final_books_Url.csv', header=None)
links = BookURLs[0].tolist()

In [ ]:
for link in links:
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        most_extract = soup.find_all("div" , class_='clearfix')
        